## Tensorboard - AND 사례
- 출처: [PinkWink](https://pinkwink.kr/1086)

In [1]:
import numpy as np
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
from tqdm import tqdm_notebook

Instructions for updating:
non-resource variables are not supported in the long term


In [2]:
# AND
x_data = np.array([[0, 0],
                  [0, 1],
                  [1, 0],
                  [1, 1]], dtype=np.float32)
y_data = np.array([[0],
                  [0],
                  [0],
                  [1]], dtype=np.float32)

In [3]:
X = tf.placeholder(tf.float32, [None, 2], name='x-input')
Y = tf.placeholder(tf.float32, [None, 1], name='y-input')

In [4]:
# W, b, hypothesis를 하나로 묶어서 보여줌
# vector는 histogram으로 보여주는데, tf.summary.histogram으로 변수를 잡아 주면 됨
with tf.name_scope("layer") as scope:    
    W = tf.Variable(tf.random_normal([2, 1]), name='weight')
    b = tf.Variable(tf.random_normal([1]), name='bias')

    hypothesis = tf.sigmoid(tf.matmul(X, W) + b)
    
    w_hist = tf.summary.histogram("weight", W)
    b_hist = tf.summary.histogram("bias", b)
    hypothesis_hist = tf.summary.histogram("hypothesis", hypothesis)

In [5]:
# Cost : Binary Cross Entropy
# scalar는 tf.summary.scalar로 변수를 잡아 주면 됨
with tf.name_scope("cost") as scope:
    cost = -tf.reduce_mean(Y * tf.log(hypothesis) + (1. - Y) * tf.log(1. - hypothesis))
    cost_summ = tf.summary.scalar("cost", cost)

In [6]:
# Optimizer : 경사하강법
with tf.name_scope("train") as scope:
    train = tf.train.GradientDescentOptimizer(learning_rate=0.1).minimize(cost)

In [7]:
# Accuracy computation
# True if hypothesis > 0.5 else False
predicted = tf.cast(hypothesis > 0.5, dtype=tf.float32)
accuracy = tf.reduce_mean(tf.cast(tf.equal(predicted, Y), dtype=tf.float32))
accuracy_summ = tf.summary.scalar("accuracy", accuracy)

In [8]:
# Launch graph
with tf.Session() as sess:
    # tensorboard --logdir=./logs/and
    merged_summary = tf.summary.merge_all()
    writer = tf.summary.FileWriter("./logs/and")
    writer.add_graph(sess.graph)  # Show the graph
    
    # Initialize TensorFlow variables
    sess.run(tf.global_variables_initializer())

    for step in tqdm_notebook(range(10001)):
        summary, _ = sess.run([merged_summary, train], feed_dict={X: x_data, Y: y_data})
        writer.add_summary(summary, global_step=step)

    # Accuracy report
    h, p, a = sess.run([hypothesis, predicted, accuracy], feed_dict={X: x_data, Y: y_data})
    print("\nHypothesis:\n", h, "\nPredicted:\n", p, "\nAccuracy: ", a)



Hypothesis:
 [[1.2451175e-05]
 [2.0240257e-02]
 [2.0240294e-02]
 [9.7165132e-01]] 
Predicted:
 [[0.]
 [0.]
 [0.]
 [1.]] 
Accuracy:  1.0
